In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from huggingface_hub import InferenceClient

# Initialize HuggingFace Inference client
client = InferenceClient("google/gemma-2-2b-it", token="paste-your-token-here")

# Define state structure
class LLMState(TypedDict):
    question: str
    answer: str

# Node function
def llm_qa(state: LLMState) -> LLMState:
    question = state["question"]
    messages = [{"role": "user", "content": question}]
    
    response = client.chat.completions.create(
        messages=messages,
        max_tokens=200,
        temperature=0.7
    )
    
    state["answer"] = response.choices[0].message["content"]
    return state

# Create the graph
graph = StateGraph(LLMState)
graph.add_node('llm_qa', llm_qa)
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

workflow = graph.compile()

# Execute workflow
initial_state = {'question': 'how far is moon from the earth'}
final_state = workflow.invoke(initial_state)
print(final_state)
